# **Iowa Liquor Retail Sales Big Query Dataset Transformation & Analysis** (Underway...)
---

**The dataset used here includes statewide wholesale liquor purchases by Iowa (state in USA) retailers since January 1, 2012. It details orders from grocery stores, liquor stores, and convenience stores, including store locations, liquor brands, sizes, and quantities. The complete Iowa state's liquor retail sales raw dataset contains more than 40 million records and can be downloaded from [this Big Query link](https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales).**

**For our data transformation and analysis activity, we've used here only the top 30000 records from the dataset.**

---

## **Importing the dataset**

In [2]:
import pandas as pd

df = pd.read_csv('/content/BQ_liquor_sales_data.csv')
pd.set_option('display.max_columns', df.shape[1])

In [2]:
df.head(3)

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,RINV-04433600124,2022-12-27,5102,WILKIE LIQUORS,724 1ST STREET NE,MOUNT VERNON,52314.0,POINT(-91.41231 41.92012),NaN,LINN,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0,-22.19
1,RINV-05415000024,2024-09-03,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627,POINT(-91.37319 40.62423),NaN,LEE,1092100.0,IMPORTED DISTILLED SPIRITS SPECIALTY,434.0,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0,-6.34
2,RINV-04846600166,2023-09-12,2560,HY-VEE FOOD STORE (1396) / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302.0,POINT(-91.572182976 42.037394006),NaN,LINN,1031100.0,AMERICAN VODKAS,301.0,FIFTH GENERATION INC,38176,TITOS HANDMADE VODKA,12,750,10.00,15.00,-24,-360.00,-18.0,-4.75


In [3]:
# Resetting the index to start from 1
df.index += 1
df.head(2)

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
1,RINV-04433600124,2022-12-27,5102,WILKIE LIQUORS,724 1ST STREET NE,MOUNT VERNON,52314.0,POINT(-91.41231 41.92012),NaN,LINN,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0,-22.19
2,RINV-05415000024,2024-09-03,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627,POINT(-91.37319 40.62423),NaN,LEE,1092100.0,IMPORTED DISTILLED SPIRITS SPECIALTY,434.0,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0,-6.34


## **Data Transformation**

**First, let's create a custom function to shift in the frame those columns that'll be created anew:**

In [4]:
# main_col : col beside which new columns will be shifted
# mover_cols_list : newly created cols that need shifting beside main_col in L2R order
# df_main : main dataframe ('df')

def shift_columns(df_main, main_col, mover_cols_list):
  all_cols_list = df_main.columns.tolist()  # Listing current df columns
  main_col_pos  = df_main.columns.get_loc(main_col)   # '.get_loc()' returns 0-based index of col in df

  for col in mover_cols_list:
    all_cols_list.insert(main_col_pos + 1, all_cols_list.pop(all_cols_list.index(col)))
    main_col_pos += 1    # Shifting the index of col pos towards right in df

  return df_main[all_cols_list]


**Some methods for premiliary checks are as follows. One can run them one line at a time.**

In [ ]:
'''Methods for preliminary checks'''

# df.shape              # o/p : (30000, 24)
# df.info()             # Shows df shape, col names, their non-null val count, & dtypes
# df.dtypes             # Col names & their dtypes
# df.describe()         # Statistical data about df's numeric cols
# df.axes               # List of row axis' and col axis' labels, in that order
# df.index              # List of labels in index col
# df.columns            # List of all col labels
# df.keys()             # List of all col labels
# df.index.name         # Label of index col
# df.index.names        # Labels of multi-col index, aka multi-index
# df.ndim               # No. of dimensions in df (2 here)
# df.memory_usage()     # Memory usage of each col in bytes
# df.select_dtypes(exclude = 'object')        # include/exclude cols of specified dtypes

**Checking dtypes of columns**

In [130]:
[df.dtypes]  # Enclosing in brackets for a compact o/p

[invoice_and_item_number     object
 date                        object
 store_number                 int64
 store_name                  object
 address                     object
 city                        object
 zip_code                    object
 store_location              object
 county_number              float64
 county                      object
 category                   float64
 category_name               object
 vendor_number              float64
 vendor_name                 object
 item_number                  int64
 item_description            object
 pack                         int64
 bottle_volume_ml             int64
 state_bottle_cost          float64
 state_bottle_retail        float64
 bottles_sold                 int64
 sale_dollars               float64
 volume_sold_liters         float64
 volume_sold_gallons        float64
 dtype: object]

**Digging deeper into the dtype of each column:**

In [ ]:
from pandas.api.types import infer_dtype
# for col in df.columns: print(col, ':', infer_dtype(df[col])) # Returns specific dtype of each col

**Resetting column names to title-case:**

In [5]:
df.columns = ['_'.join(word.title() for word in col.split('_')) for col in df.columns]
df.columns

Index(['Invoice_And_Item_Number', 'Date', 'Store_Number', 'Store_Name',
       'Address', 'City', 'Zip_Code', 'Store_Location', 'County_Number',
       'County', 'Category', 'Category_Name', 'Vendor_Number', 'Vendor_Name',
       'Item_Number', 'Item_Description', 'Pack', 'Bottle_Volume_Ml',
       'State_Bottle_Cost', 'State_Bottle_Retail', 'Bottles_Sold',
       'Sale_Dollars', 'Volume_Sold_Liters', 'Volume_Sold_Gallons'],
      dtype='object')

In [6]:
df.rename(columns = {'Invoice_And_Item_Number' : 'Bill_Number', 'Date' : 'Order_Date'}, inplace = True)
df.columns[0:2]

Index(['Bill_Number', 'Order_Date'], dtype='object')

**The** 'date' **column's dtype should be of** datetime **type so that we can extract various date components later on.**

In [7]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format = '%Y-%m-%d')
df['Order_Date'].dtype

dtype('<M8[ns]')

**Extracting date components from** 'Order_Date' **column into separate columns & shifting them beside** Order_Date **column:**

In [41]:
# Columns' creation
df['Order_Year'] = df['Order_Date'].dt.year
df['Order_Month'] = df['Order_Date'].dt.month
df['Order_Day'] = df['Order_Date'].dt.day
df['Order_Quarter'] = df['Order_Date'].dt.quarter

# Shifting of columns (calling custom fn created at the top of this section)
df = shift_columns(df, 'Order_Date', ['Order_Year', 'Order_Month', 'Order_Day', 'Order_Quarter'])

pd.set_option('display.max_columns', df.shape[1])
df.head(2)

,Bill_Number,Order_Date,Order_Year,Order_Month,Order_Day,Order_Quarter,Store_Number,Store_Name,Address,City,Zip_Code,Store_Location,County_Number,County,Category,Category_Name,Vendor_Number,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_Ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons
1,RINV-04433600124,2022-12-27,2022,12,27,4,5102,WILKIE LIQUORS,724 1ST STREET NE,MOUNT VERNON,52314.0,POINT(-91.41231 41.92012),NaN,LINN,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0,-22.19
2,RINV-05415000024,2024-09-03,2024,9,3,3,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627,POINT(-91.37319 40.62423),NaN,LEE,1092100.0,IMPORTED DISTILLED SPIRITS SPECIALTY,434.0,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0,-6.34


**Splitting column** Store_Location **into two columns:** Latitude **and** Longitude**.**

In [42]:
# 'strip()' crops all chars of 'POINT()' from a string's ends
df['Store_Location'] = df['Store_Location'].str.strip('POINT()')
df['Store_Location'].head(3)

,Store_Location
1,-91.41231 41.92012
2,-91.37319 40.62423
3,-91.572182976 42.037394006


In [43]:
# Splitting vals into columns through 'expand' property
df[['Longitude', 'Latitude']] = df['Store_Location'].str.split(' ', expand = True)
df[['Longitude', 'Latitude']].head(2)

,Longitude,Latitude
1,-91.41231,41.92012
2,-91.37319,40.62423


**Shifting the above two newly-created columns beside column** Store_Location **:**

In [44]:
# Calling custom fn defined at the top earlier to shift cols

df = shift_columns(df, 'Store_Location', ['Longitude', 'Latitude'])

pd.set_option('display.max_columns', df.shape[0])
df.head(2)

,Bill_Number,Order_Date,Order_Year,Order_Month,Order_Day,Order_Quarter,Store_Number,Store_Name,Address,City,Zip_Code,Store_Location,Longitude,Latitude,County_Number,County,Category,Category_Name,Vendor_Number,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_Ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons
1,RINV-04433600124,2022-12-27,2022,12,27,4,5102,WILKIE LIQUORS,724 1ST STREET NE,MOUNT VERNON,52314.0,-91.41231 41.92012,-91.41231,41.92012,NaN,LINN,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0,-22.19
2,RINV-05415000024,2024-09-03,2024,9,3,3,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627,-91.37319 40.62423,-91.37319,40.62423,NaN,LEE,1092100.0,IMPORTED DISTILLED SPIRITS SPECIALTY,434.0,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0,-6.34


**Identifying the number of unique values in each column**

In [ ]:
# [df.nunique()]  # Enclosing in brackets for a compact o/p

**Dropping unwanted columns**

In [45]:
df.drop(columns = ['Address', 'Zip_Code', 'Category', 'Vendor_Number', 'Volume_Sold_Gallons'], inplace = True)
df.head(2)

,Bill_Number,Order_Date,Order_Year,Order_Month,Order_Day,Order_Quarter,Store_Number,Store_Name,City,Store_Location,Longitude,Latitude,County_Number,County,Category_Name,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_Ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters
1,RINV-04433600124,2022-12-27,2022,12,27,4,5102,WILKIE LIQUORS,MOUNT VERNON,-91.41231 41.92012,-91.41231,41.92012,NaN,LINN,CANADIAN WHISKIES,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0
2,RINV-05415000024,2024-09-03,2024,9,3,3,3549,QUICKER LIQUOR STORE,FORT MADISON,-91.37319 40.62423,-91.37319,40.62423,NaN,LEE,IMPORTED DISTILLED SPIRITS SPECIALTY,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0


**Checking the number of nulls in each column:**

In [39]:
[df.isnull().sum()]  # Enclosing in brackets for a compact o/p

[Bill_Number               0
 Order_Date                0
 Order_Year                0
 Order_Month               0
 Order_Day                 0
 Order_Quarter             0
 Store_Number              0
 Store_Name                0
 City                      0
 Store_Location         2056
 Longitude              2056
 Latitude               2056
 County_Number          8248
 County                   76
 Category_Name            22
 Vendor_Name               0
 Item_Number               0
 Item_Description          0
 Pack                      0
 Bottle_Volume_Ml          0
 State_Bottle_Cost         1
 State_Bottle_Retail       1
 Bottles_Sold              0
 Sale_Dollars              1
 Volume_Sold_Liters        0
 dtype: int64]

In [38]:
df[df['County_Number'].isnull()]

,Bill_Number,Order_Date,Order_Year,Order_Month,Order_Day,Order_Quarter,Store_Number,Store_Name,City,Store_Location,Longitude,Latitude,County_Number,County,Category_Name,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_Ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters
1,RINV-04433600124,2022-12-27,2022,12,27,4,5102,WILKIE LIQUORS,MOUNT VERNON,-91.41231 41.92012,-91.41231,41.92012,NaN,LINN,CANADIAN WHISKIES,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.00
2,RINV-05415000024,2024-09-03,2024,9,3,3,3549,QUICKER LIQUOR STORE,FORT MADISON,-91.37319 40.62423,-91.37319,40.62423,NaN,LEE,IMPORTED DISTILLED SPIRITS SPECIALTY,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.00
3,RINV-04846600166,2023-09-12,2023,9,12,3,2560,HY-VEE FOOD STORE (1396) / MARION,MARION,-91.572182976 42.037394006,-91.572182976,42.037394006,NaN,LINN,AMERICAN VODKAS,FIFTH GENERATION INC,38176,TITOS HANDMADE VODKA,12,750,10.00,15.00,-24,-360.00,-18.00
4,RINV-04681700152,2023-06-09,2023,6,9,2,2663,HY-VEE FOOD STORE / URBANDALE,URBANDALE,-93.738906371 41.630504742,-93.738906371,41.630504742,NaN,POLK,AMERICAN DISTILLED SPIRITS SPECIALTY,INFINIUM SPIRITS,86439,SKREWBALL PEANUT BUTTER WHISKEY,12,750,18.50,27.75,-24,-666.00,-18.00
5,RINV-04746500069,2023-07-12,2023,7,12,3,4346,ROY'S FOODLAND,SHELLSBURG,-91.868949035 42.094179011,-91.868949035,42.094179011,NaN,BENTON,CANADIAN WHISKIES,DIAGEO AMERICAS,10808,CROWN ROYAL REGAL APPLE,12,1000,20.49,30.74,-12,-368.88,-12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29977,INV-59623700038,2023-06-21,2023,6,21,2,2614,HY-VEE #3 FOOD & DRUGSTORE / DAVENPORT,DAVENPORT,-90.54879501 41.554922984,-90.54879501,41.554922984,NaN,SCOTT,WHITE RUM,BACARDI USA INC,43124,BACARDI SUPERIOR,24,375,5.00,7.50,3,22.50,1.12
29984,INV-56711800089,2023-03-15,2023,3,15,1,5315,BROTHER'S MARKET WINE AND SPIRITS,SIGOURNEY,-92.20517 41.33328,-92.20517,41.33328,NaN,KEOKUK,WHITE RUM,BACARDI USA INC,43124,BACARDI SUPERIOR,24,375,5.00,7.50,3,22.50,1.12
29985,INV-76920900081,2024-11-26,2024,11,26,4,2572,HY-VEE FOOD STORE / CEDAR FALLS,CEDAR FALLS,-92.43651 42.51125,-92.43651,42.51125,NaN,BLACK HAWK,WHITE RUM,BACARDI USA INC,43124,BACARDI SUPERIOR,24,375,5.00,7.50,3,22.50,1.12
29987,INV-74431900031,2024-09-18,2024,9,18,3,4793,FORT MADISON FAST BREAK,FORT MADISON,-91.377328034 40.623500003,-91.377328034,40.623500003,NaN,LEE,WHITE RUM,BACARDI USA INC,43124,BACARDI SUPERIOR,24,375,5.00,7.50,3,22.50,1.12


**Since** City **column has no nulls, we can use its values to identify values in** County_Number **column that are  presently NaN.**

In [ ]:
# sorted(df['County_Number'].unique())

# All cities having 'County_Number = NaN'
city_list = df[df['County_Number'].isnull()]['City'].unique()
# city_list


In [ ]:
!pip install "camelot-py[base]" # Install 'camelot-py' with base dependencies

In [10]:
import camelot

# Extract tables from all 19 pages
tables = camelot.read_pdf('/content/iowaCityCounty.pdf', pages = '1-19')

print(f"Total tables extracted: {tables.n}") # Check how many tables were detected

dfs = [table.df for table in tables]  # Convert each table to DataFrame
final_df = pd.concat(dfs, ignore_index=True)  # Merge all tables

final_df.head() # Show the first few rows

# Save to CSV (Optional)
# final_df.to_csv("extracted_data.csv", index=False)


Total tables extracted: 19


,0,1,2
0,Name,Date of Incorporation,County
1,Ackley,"August 28, 1869",Hardin
2,Ackworth,"May 9, 1881",Warren
3,Adair,"February 23, 1884",Adair
4,Adel,"June 27, 1877",Dallas


**Promoting first row to column headers:**

In [14]:
final_df.columns = final_df.iloc[0]  # Set first row as column headers
final_df.head(3)

,Name,Date of Incorporation,County
0,Name,Date of Incorporation,County
1,Ackley,"August 28, 1869",Hardin
2,Ackworth,"May 9, 1881",Warren


**Removing 1st row from above o/p:**

In [15]:
final_df = final_df[1:].reset_index(drop=True)
final_df.head(2)

,Name,Date of Incorporation,County
0,Ackley,"August 28, 1869",Hardin
1,Ackworth,"May 9, 1881",Warren


**Dropping column, renaming remaining columns & changing all values to upper case:**

In [22]:
final_df = final_df.drop(columns = 'Date of Incorporation')  # Dropping unwanted column
final_df = final_df.rename(columns = {'Name' : 'CityName', 'County' : 'CityCounty'})
final_df = final_df.map(lambda x: x.upper() if isinstance(x, str) else x)  # All vals in frame to upper case

final_df.head(2)

,CityName,CityCounty
0,ACKLEY,HARDIN
1,ACKWORTH,WARREN


In [28]:
final_df.set_index('CityName', inplace = True)
final_df.head(3)

,CityCounty
CityName,
ACKLEY,HARDIN
ACKWORTH,WARREN
ADAIR,ADAIR


In [46]:
final_df.loc[df['City']]

KeyError: '[\'DEWITT\', \'AMANA\', \'WASHBURN\', \'JEWELL\', "ARNOLD\'S PARK", \'CLEARLAKE\', \'MT PLEASANT\', nan, \'ST CHARLES\', \'GRAND MOUNDS\', \'LONETREE\', \'OTTUWMA\', \'ST ANSGAR\', \'LEMARS\', \'LECLAIRE\', \'KELLOG\', \'MT VERNON\', \'DENMARK\', \'GUTTENBURG\'] not in index'

**We had to download .CSV file of Iowa county's and their numbers [from this website](https://www.downloadexcelfiles.com/us_en/download-excel-file-list-counties-iowa#gsc.tab=0).**

**The redundant columns from that file were dropped and all county names were changed to uppercase.**

In [33]:
df_county_numbers = pd.read_csv('/content/iowaCountyNumbers.csv')

df_county_numbers.set_index('County', inplace = True)

df_county_numbers.head(3)

,CountyNumber
County,
ADAIR,1
ADAMS,2
ALLAMAKEE,3


In [ ]:
df['County_Number']

**Identifying nulls in rows:**

In [34]:
# df.isnull().sum(axis=1).max()  # o/p: 6   ;  Max no. of nulls in a ROW is 6 i.e. ~26% of data in the row is absent.
rows_nulls_indexes = df.loc[df.isnull().sum(axis=1) == 6].index  # Extracting the indices of ROWS having nulls = 6 in them
rows_nulls_indexes

Index([ 3466,  6231,  6359,  6525,  6669,  6945,  6946,  7709,  8525,  8677,
       ...
       27579, 27653, 27676, 27904, 27928, 28736, 28824, 28924, 28926, 29490],
      dtype='int64', length=114)

**Rows at these ↑ 114 indices have ~26% of the data absent and can be dropped from dataset without much affecting our analysis later on.**

In [35]:
df.drop(rows_nulls_indexes, inplace = True)

**Identifying duplicated records:**

In [36]:
df[df.duplicated()].index  # Index list in o/p is empty implying there are no duplicated records

Index([], dtype='int64')

In [186]:
from shapely.geometry import Point

# Create GeoSeries from latitude and longitude
df['Geo_Location'] = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]

pd.set_option('display.max_columns', df.shape[1])
df.head(3)

,Bill_Number,Order_Date,Order_Year,Order_Month,Order_Day,Order_Quarter,Store_Number,Store_Name,Address,City,Zip_Code,Store_Location,County_Number,County,Category,Category_Name,Vendor_Number,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_Ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons,Longitude,Latitude,Geo_Location
1,RINV-04433600124,2022-12-27,2022,12,27,4,5102,WILKIE LIQUORS,724 1ST STREET NE,MOUNT VERNON,52314.0,-91.41231 41.92012,NaN,LINN,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL,12,1000,19.99,29.99,-84,-2519.16,-84.0,-22.19,-91.41231,41.92012,POINT (-91.41231 41.92012)
2,RINV-05415000024,2024-09-03,2024,9,3,3,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627,-91.37319 40.62423,NaN,LEE,1092100.0,IMPORTED DISTILLED SPIRITS SPECIALTY,434.0,LUXCO INC,75087,JUAREZ GOLD DSS,12,1000,5.09,7.64,-24,-183.36,-24.0,-6.34,-91.37319,40.62423,POINT (-91.37319 40.62423)
3,RINV-04846600166,2023-09-12,2023,9,12,3,2560,HY-VEE FOOD STORE (1396) / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302.0,-91.572182976 42.037394006,NaN,LINN,1031100.0,AMERICAN VODKAS,301.0,FIFTH GENERATION INC,38176,TITOS HANDMADE VODKA,12,750,10.00,15.00,-24,-360.00,-18.0,-4.75,-91.572182976,42.037394006,POINT (-91.572182976 42.037394006)


In [ ]:
!pip install geopandas
!wget https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_1_states_provinces.zip
!unzip ne_110m_admin_1_states_provinces.zip



In [201]:
import geopandas as gpd, matplotlib.pyplot as plt
from shapely.geometry import Point

data = {'latitude': df['Latitude'], 'longitude': df['Longitude']}
df2 = pd.DataFrame(data)

# Create geometry column
df2['geometry'] = df2.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df2, geometry='geometry')

# Load the states shapefile
states = gpd.read_file("ne_110m_admin_1_states_provinces.shp")

# Filter for Iowa
iowa = states[states.name == "Iowa"]

# Plot the map of Iowa
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust figure size as needed
iowa.plot(ax=ax, color='lightblue', edgecolor='black')

# Plot the geographic points on top
gdf.plot(ax=ax, color='red', markersize=50)

# Set title and labels
ax.set_title("Geographic Points in Iowa", fontsize=16)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Show the plot
plt.show()

KeyError: 'Latitude'

## **Data Analysis**

### Filter all records of whisky (or whiskey or scotch) sales post Jan 2024.

In [104]:
def check_whisky(y):    # Fn to check if val in 'item_description' col contains substrings 'whisky', 'whiskey', or 'scotch'
  return any(word in y.lower() for word in ['whisky', 'whiskey', 'scotch'])

df[df['item_description'].apply(check_whisky) & (df['date'] > 'Jan-2024')][['date', 'item_description', 'volume_sold_liters', 'sale_dollars']]

,date,item_description,volume_sold_liters,sale_dollars
11,2024-01-17,FIREBALL CINNAMON WHISKEY,-9.60,-144.00
25,2024-10-04,FIREBALL CINNAMON WHISKEY,-3.00,-54.00
65,2024-07-01,FIREBALL CINNAMON WHISKEY PARTY BUCKET,0.05,77.40
66,2024-03-25,FIREBALL CINNAMON WHISKEY PARTY BUCKET,0.05,77.40
70,2025-01-29,FIREBALL CINNAMON WHISKEY PARTY BUCKET,0.05,78.60
...,...,...,...,...
28520,2024-01-29,FIREBALL CINNAMON WHISKEY,1.00,18.00
28521,2024-07-31,FIREBALL CINNAMON WHISKEY,1.00,18.00
28827,2024-04-03,CLAN MACGREGOR SCOTCH,1.00,9.99
29348,2024-06-26,SKREWBALL PEANUT BUTTER WHISKEY,1.00,28.47
